In [1]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import clear_output
%matplotlib inline

In [2]:
from math import floor
import numpy as np

def row_col_to_seq(row_col, num_cols):  #Converts state number to row_column format
    return row_col[:,0] * num_cols + row_col[:,1]

def seq_to_col_row(seq, num_cols): #Converts row_column format to state number
    r = floor(seq / num_cols)
    c = seq - r * num_cols
    return np.array([[r, c]])

class GridWorld:
    """
    Creates a gridworld object to pass to an RL algorithm.
    Parameters
    ----------
    num_rows : int
        The number of rows in the gridworld.
    num_cols : int
        The number of cols in the gridworld.
    start_state : numpy array of shape (1, 2), np.array([[row, col]])
        The start state of the gridworld (can only be one start state)
    goal_states : numpy arrany of shape (n, 2)
        The goal states for the gridworld where n is the number of goal
        states.
    """
    def __init__(self, num_rows, num_cols, start_state, goal_states, wind = False):
        self.num_rows = num_rows
        self.num_cols = num_cols
        self.start_state = start_state
        self.goal_states = goal_states
        self.obs_states = None
        self.bad_states = None
        self.num_bad_states = 0
        self.p_good_trans = None
        self.bias = None
        self.r_step = None
        self.r_goal = None
        self.r_dead = None
        self.gamma = 1 # default is no discounting
        self.wind = wind

    def add_obstructions(self, obstructed_states=None, bad_states=None, restart_states=None):

        self.obs_states = obstructed_states
        self.bad_states = bad_states
        if bad_states is not None:
            self.num_bad_states = bad_states.shape[0]
        else:
            self.num_bad_states = 0
        self.restart_states = restart_states
        if restart_states is not None:
            self.num_restart_states = restart_states.shape[0]
        else:
            self.num_restart_states = 0

    def add_transition_probability(self, p_good_transition, bias):

        self.p_good_trans = p_good_transition
        self.bias = bias

    def add_rewards(self, step_reward, goal_reward, bad_state_reward=None, restart_state_reward = None):

        self.r_step = step_reward
        self.r_goal = goal_reward
        self.r_bad = bad_state_reward
        self.r_restart = restart_state_reward


    def create_gridworld(self):

        self.num_actions = 4
        self.num_states = self.num_cols * self.num_rows# +1
        self.start_state_seq = row_col_to_seq(self.start_state, self.num_cols)
        self.goal_states_seq = row_col_to_seq(self.goal_states, self.num_cols)

        # rewards structure
        self.R = self.r_step * np.ones((self.num_states, 1))
        #self.R[self.num_states-1] = 0
        self.R[self.goal_states_seq] = self.r_goal

        for i in range(self.num_bad_states):
            if self.r_bad is None:
                raise Exception("Bad state specified but no reward is given")
            bad_state = row_col_to_seq(self.bad_states[i,:].reshape(1,-1), self.num_cols)
            #print("bad states", bad_state)
            self.R[bad_state, :] = self.r_bad
        for i in range(self.num_restart_states):
            if self.r_restart is None:
                raise Exception("Restart state specified but no reward is given")
            restart_state = row_col_to_seq(self.restart_states[i,:].reshape(1,-1), self.num_cols)
            #print("restart_state", restart_state)
            self.R[restart_state, :] = self.r_restart

        # probability model
        if self.p_good_trans == None:
            raise Exception("Must assign probability and bias terms via the add_transition_probability method.")

        self.P = np.zeros((self.num_states,self.num_states,self.num_actions))
        for action in range(self.num_actions):
            for state in range(self.num_states):


                # check if the state is the goal state or an obstructed state - transition to end
                row_col = seq_to_col_row(state, self.num_cols)
                if self.obs_states is not None:
                    end_states = np.vstack((self.obs_states, self.goal_states))
                else:
                    end_states = self.goal_states

                if any(np.sum(np.abs(end_states-row_col), 1) == 0):
                    self.P[state, state, action] = 1

                # else consider stochastic effects of action
                else:
                    for dir in range(-1,2,1):

                        direction = self._get_direction(action, dir)
                        next_state = self._get_state(state, direction)
                        if dir == 0:
                            prob = self.p_good_trans
                        elif dir == -1:
                            prob = (1 - self.p_good_trans)*(self.bias)
                        elif dir == 1:
                            prob = (1 - self.p_good_trans)*(1-self.bias)

                        self.P[state, next_state, action] += prob

                # make restart states transition back to the start state with
                # probability 1
                if self.restart_states is not None:
                    if any(np.sum(np.abs(self.restart_states-row_col),1)==0):
                        next_state = row_col_to_seq(self.start_state, self.num_cols)
                        self.P[state,:,:] = 0
                        self.P[state,next_state,:] = 1
        return self

    def _get_direction(self, action, direction):

        left = [2,3,1,0]
        right = [3,2,0,1]
        if direction == 0:
            new_direction = action
        elif direction == -1:
            new_direction = left[action]
        elif direction == 1:
            new_direction = right[action]
        else:
            raise Exception("getDir received an unspecified case")
        return new_direction

    def _get_state(self, state, direction):

        row_change = [-1,1,0,0]
        col_change = [0,0,-1,1]
        row_col = seq_to_col_row(state, self.num_cols)
        row_col[0,0] += row_change[direction]
        row_col[0,1] += col_change[direction]

        # check for invalid states
        if self.obs_states is not None:
            if (np.any(row_col < 0) or
                np.any(row_col[:,0] > self.num_rows-1) or
                np.any(row_col[:,1] > self.num_cols-1) or
                np.any(np.sum(abs(self.obs_states - row_col), 1)==0)):
                next_state = state
            else:
                next_state = row_col_to_seq(row_col, self.num_cols)[0]
        else:
            if (np.any(row_col < 0) or
                np.any(row_col[:,0] > self.num_rows-1) or
                np.any(row_col[:,1] > self.num_cols-1)):
                next_state = state
            else:
                next_state = row_col_to_seq(row_col, self.num_cols)[0]

        return next_state

    def reset(self):
      return int(self.start_state_seq)

    def step(self, state, action):
        p, r = 0, np.random.random()
        for next_state in range(self.num_states):

            p += self.P[state, next_state, action]

            if r <= p:
                break

        if(self.wind and np.random.random() < 0.4):

          arr = self.P[next_state, :, 3]
          next_next = np.where(arr == np.amax(arr))
          next_next = next_next[0][0]
          return next_next, self.R[next_next]
        else:
          return next_state, self.R[next_state]


In [3]:
action_space = [0, 1, 2, 3]   #[UP, DOWN, LEFT, RIGHT]

### Exploration Strategies
 1. Epsilon - greedy
 2. Softmax

In [4]:
seed = 42
rg = np.random.RandomState(seed)

# Epsilon - greedy
def choose_action_epsilon_greedy(Q, state, epsilon): #state must be an array of riw_index and column_index
  if not Q[state[0], state[1]].any() or rg.rand() < epsilon:
    return rg.choice(action_space, size = (1, ))[0]
  else:
    return np.argmax(Q[state[0], state[1]])

# Softmax
def choose_action_softmax(Q, state, tau):
  p = [None for _ in range(len(action_space))]
  for i in range(len(action_space)):
    p[i] = min(np.exp((Q[state[0], state[1], i]) / tau),1000)
  p /= np.sum(p)
  return rg.choice(action_space, size = (1, ), p = p)[0]

### SARSA Algorithm Definition


In [5]:
def sarsa(gw, Q, tau0, epsilon0, alpha0, gamma0, episodes = 10000, choose_action = choose_action_softmax):

  episode_rewards = np.zeros(episodes)
  steps_to_completion = np.zeros(episodes)
  state_visit_count = np.zeros((gw.num_rows, gw.num_cols))

  parameter = tau0
  alpha = alpha0
  gamma = gamma0
  if choose_action == choose_action_epsilon_greedy:
    parameter = epsilon0

  for ep in tqdm(range(episodes)):
    tot_reward, steps = 0, 0

    state = gw.reset()
    state_rc = seq_to_col_row(state, gw.num_cols)
    action = choose_action(Q, state_rc[0], parameter)
    done = False
    while not done:

      if any(np.array_equal(state_rc[0], state) for state in gw.goal_states) or steps >= 100:
        done = True
        break

      next_state, reward = gw.step(state, action)
      next_state_rc = seq_to_col_row(next_state, gw.num_cols)

      next_action = choose_action(Q, next_state_rc[0], parameter)
      Q[state_rc[0][0], state_rc[0][1],  action] += alpha * (reward + gamma * Q[next_state_rc[0][0], next_state_rc[0][1], next_action] - Q[state_rc[0][0], state_rc[0][1],  action])

      state_visit_count[state_rc[0][0], state_rc[0][1]] += 1
      tot_reward += reward
      steps += 1

      state, state_rc, action = next_state, next_state_rc, next_action



    episode_rewards[ep] = tot_reward
    steps_to_completion[ep] = steps
  return Q, episode_rewards, steps_to_completion

In [6]:
# specify world parameters
num_cols = 10
num_rows = 10
obstructions = np.array([[0,7],[1,1],[1,2],[1,3],[1,7],[2,1],[2,3],              #All obstruction states
                         [2,7],[3,1],[3,3],[3,5],[4,3],[4,5],[4,7],
                         [5,3],[5,7],[5,9],[6,3],[6,9],[7,1],[7,6],
                         [7,7],[7,8],[7,9],[8,1],[8,5],[8,6],[9,1]])
bad_states = np.array([[1,9],[4,2],[4,4],[7,5],[9,9]])                           #All bad states
restart_states = np.array([[3,7],[8,2]])                                         #All restart states
goal_states = np.array([[0,9],[2,2],[8,7]])

In [7]:
start_state = np.array([[0, 4]])
# create model
gw = GridWorld(num_rows = num_rows, num_cols = num_cols, start_state = start_state, goal_states = goal_states, wind = True)
gw.add_obstructions(obstructed_states = obstructions, bad_states = bad_states, restart_states = restart_states)
gw.add_rewards(step_reward = - 1, goal_reward = 10, bad_state_reward = - 6, restart_state_reward = - 100)
gw.add_transition_probability(p_good_transition = 1.0, bias = 0.5)
env = gw.create_gridworld()

In [8]:
start_state = np.array([[0, 4]])
alpha_list = [0.01, 0.05, 0.1, 0.2]
gamma_list = [0.8, 0.9, 1.0]
tau_list = [1.0, 2.0, 4.0, 10.0]
epsilon_list = [0.01, 0.05, 0.1]

store_tau = []
#store_epsilon = []

#For Soft Max
best_tau, best_tau_alpha, best_tau_gamma = tau_list[0], alpha_list[0], gamma_list[0]

Q = np.zeros((gw.num_rows, gw.num_cols, len(action_space)))

Q, rewards, steps = sarsa(gw, Q, best_tau, epsilon_list[0], best_tau_alpha, best_tau_gamma, episodes = 5000)
best_tau_reward = np.mean(rewards)
for tau in tau_list:
  for alpha in alpha_list:
    for gamma in gamma_list:
      Q = np.zeros((gw.num_rows, gw.num_cols, len(action_space)))
      Q, rewards, steps = sarsa(gw, Q, tau, epsilon_list[0], alpha, gamma, episodes = 5000)
      store_tau.append({"tau" : tau, "alpha" : alpha, "gamma" : gamma, "reward" : np.mean(rewards)})
      if np.mean(rewards) > best_tau_reward:
        best_tau = tau
        best_tau_reward = np.mean(rewards)
        best_tau_alpha = alpha
        best_tau_gamma = gamma
          
print(best_tau, best_tau_alpha, best_tau_gamma, best_tau_reward)

  0%|                                                                                         | 0/5000 [00:00<?, ?it/s]C:\Users\kanni\AppData\Local\Temp\ipykernel_2796\2297487967.py:180: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return int(self.start_state_seq)
C:\Users\kanni\AppData\Local\Temp\ipykernel_2796\12305718.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Q[state_rc[0][0], state_rc[0][1],  action] += alpha * (reward + gamma * Q[next_state_rc[0][0], next_state_rc[0][1], next_action] - Q[state_rc[0][0], state_rc[0][1],  action])
C:\Users\kanni\AppData\Local\Temp\ipykernel_2796\12305718.py:40: DeprecationWarning: Conversio

1.0 0.2 1.0 -14.1132


In [9]:
start_state = np.array([[0, 4]])
alpha_list = [0.01, 0.05, 0.1, 0.2]
gamma_list = [0.8, 0.9, 1.0]
tau_list = [1.0, 2.0, 4.0, 10.0]
epsilon_list = [0.01, 0.05, 0.1]

#store_tau = []
store_epsilon = []

# For Epsilon Greedy
best_epsilon, best_epsilon_alpha, best_epsilon_gamma = epsilon_list[0], alpha_list[0], gamma_list[0]

Q = np.zeros((gw.num_rows, gw.num_cols, len(action_space)))

Q, rewards, steps = sarsa(gw, Q, tau_list[0], best_epsilon, best_epsilon_alpha, best_epsilon_gamma, episodes = 5000, choose_action = choose_action_epsilon_greedy)
best_epsilon_reward = np.mean(rewards)
for epsilon in epsilon_list:
  for alpha in alpha_list:
    for gamma in gamma_list:
      Q = np.zeros((gw.num_rows, gw.num_cols, len(action_space)))
      Q, rewards, steps = sarsa(gw, Q, tau_list[0], epsilon, alpha, gamma, episodes = 5000, choose_action = choose_action_epsilon_greedy)
      store_epsilon.append({"epsilon" : epsilon, "alpha" : alpha, "gamma" : gamma, "reward" : np.mean(rewards)})
      if np.mean(rewards) > best_epsilon_reward:
        best_epsilon = epsilon
        best_epsilon_reward = np.mean(rewards)
        best_epsilon_alpha = alpha
        best_epsilon_gamma = gamma
          
print(best_epsilon, best_epsilon_alpha, best_epsilon_gamma, best_epsilon_reward)

  0%|                                                                                         | 0/5000 [00:00<?, ?it/s]C:\Users\kanni\AppData\Local\Temp\ipykernel_2796\2297487967.py:180: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return int(self.start_state_seq)
C:\Users\kanni\AppData\Local\Temp\ipykernel_2796\12305718.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Q[state_rc[0][0], state_rc[0][1],  action] += alpha * (reward + gamma * Q[next_state_rc[0][0], next_state_rc[0][1], next_action] - Q[state_rc[0][0], state_rc[0][1],  action])
C:\Users\kanni\AppData\Local\Temp\ipykernel_2796\12305718.py:40: DeprecationWarning: Conversio

0.01 0.2 0.9 -9.4132


In [16]:
print(store_tau)

[{'tau': 1.0, 'alpha': 0.01, 'gamma': 0.8, 'reward': -100.7972}, {'tau': 1.0, 'alpha': 0.01, 'gamma': 0.9, 'reward': -94.0798}, {'tau': 1.0, 'alpha': 0.01, 'gamma': 1.0, 'reward': -41.415}, {'tau': 1.0, 'alpha': 0.05, 'gamma': 0.8, 'reward': -99.9524}, {'tau': 1.0, 'alpha': 0.05, 'gamma': 0.9, 'reward': -88.993}, {'tau': 1.0, 'alpha': 0.05, 'gamma': 1.0, 'reward': -17.8542}, {'tau': 1.0, 'alpha': 0.1, 'gamma': 0.8, 'reward': -99.8018}, {'tau': 1.0, 'alpha': 0.1, 'gamma': 0.9, 'reward': -88.026}, {'tau': 1.0, 'alpha': 0.1, 'gamma': 1.0, 'reward': -15.869}, {'tau': 1.0, 'alpha': 0.2, 'gamma': 0.8, 'reward': -99.7736}, {'tau': 1.0, 'alpha': 0.2, 'gamma': 0.9, 'reward': -86.0236}, {'tau': 1.0, 'alpha': 0.2, 'gamma': 1.0, 'reward': -14.1132}, {'tau': 2.0, 'alpha': 0.01, 'gamma': 0.8, 'reward': -101.618}, {'tau': 2.0, 'alpha': 0.01, 'gamma': 0.9, 'reward': -100.8822}, {'tau': 2.0, 'alpha': 0.01, 'gamma': 1.0, 'reward': -59.0362}, {'tau': 2.0, 'alpha': 0.05, 'gamma': 0.8, 'reward': -100.327},

In [10]:
print(store_epsilon)

[{'epsilon': 0.01, 'alpha': 0.01, 'gamma': 0.8, 'reward': -26.704}, {'epsilon': 0.01, 'alpha': 0.01, 'gamma': 0.9, 'reward': -25.201}, {'epsilon': 0.01, 'alpha': 0.01, 'gamma': 1.0, 'reward': -25.1484}, {'epsilon': 0.01, 'alpha': 0.05, 'gamma': 0.8, 'reward': -12.2494}, {'epsilon': 0.01, 'alpha': 0.05, 'gamma': 0.9, 'reward': -11.9916}, {'epsilon': 0.01, 'alpha': 0.05, 'gamma': 1.0, 'reward': -11.958}, {'epsilon': 0.01, 'alpha': 0.1, 'gamma': 0.8, 'reward': -10.5688}, {'epsilon': 0.01, 'alpha': 0.1, 'gamma': 0.9, 'reward': -10.2752}, {'epsilon': 0.01, 'alpha': 0.1, 'gamma': 1.0, 'reward': -10.345}, {'epsilon': 0.01, 'alpha': 0.2, 'gamma': 0.8, 'reward': -9.5266}, {'epsilon': 0.01, 'alpha': 0.2, 'gamma': 0.9, 'reward': -9.4132}, {'epsilon': 0.01, 'alpha': 0.2, 'gamma': 1.0, 'reward': -9.7422}, {'epsilon': 0.05, 'alpha': 0.01, 'gamma': 0.8, 'reward': -27.9264}, {'epsilon': 0.05, 'alpha': 0.01, 'gamma': 0.9, 'reward': -26.7666}, {'epsilon': 0.05, 'alpha': 0.01, 'gamma': 1.0, 'reward': -26

In [11]:
import pandas as pd

In [12]:
epsilon_store = [None for _ in range(len(store_epsilon))]
alpha_epsilon_store = [None for _ in range(len(store_epsilon))]
gamma_epsilon_store = [None for _ in range(len(store_epsilon))]
reward_epsilon_store = [None for _ in range(len(store_epsilon))]
for i in range(len(store_epsilon)):
    epsilon_store[i] = store_epsilon[i]['epsilon']
    alpha_epsilon_store[i] = store_epsilon[i]['alpha']
    gamma_epsilon_store[i] = store_epsilon[i]['gamma']
    reward_epsilon_store[i] = store_epsilon[i]['reward']

In [13]:
EXP_3_epsilon = pd.DataFrame({'Epsilon' : epsilon_store, 'Alpha' : alpha_epsilon_store, 'Gamma' : gamma_epsilon_store, 'Reward' : reward_epsilon_store})
EXP_3_epsilon.to_csv("EXP_3_epsilon.csv", index = False)

In [14]:
tau_store = [None for _ in range(len(store_tau))]
alpha_tau_store = [None for _ in range(len(store_tau))]
gamma_tau_store = [None for _ in range(len(store_tau))]
reward_tau_store = [None for _ in range(len(store_tau))]
for i in range(len(store_tau)):
    tau_store[i] = store_tau[i]['tau']
    alpha_tau_store[i] = store_tau[i]['alpha']
    gamma_tau_store[i] = store_tau[i]['gamma']
    reward_tau_store[i] = store_tau[i]['reward']

In [15]:
EXP_3_tau = pd.DataFrame({'Tau' : tau_store, 'Alpha' : alpha_tau_store, 'Gamma' : gamma_tau_store, 'Reward' : reward_tau_store})
EXP_3_tau.to_csv("EXP_3_tau.csv", index = False)